# NTT Mobile Spacial Statistics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid', palette='husl')
import time
from datetime import datetime

consumerKey = 'hnuzMJ1QGE3TcjlYiXohfA6vkGb55OT9xKtTCcd31L4'

### 仮説
慶應義塾大学の学生はゴールデンウィークを過ぎると登校しなくなるもしくは遅刻するひとの割合が増える。

この分析のゴールはゴールデンウィーク前後に慶應義塾大学の三大主要キャンパス(三田、日吉、SFC)における生徒の数、および登校する時間帯に変化があるかをデータを通して確かめることである。データは東京交通オープンデータチャレンジにおいてAgoop社が提供するメッシュ型人工流動データである。

2019年のゴールデンウィークは`4月27日`から`5月6日`である。取得するデータはこの一週間前のものと一週間後のもの、つまり`4月22日`\~`4月26日`と`5月7日`\~`5月10日`の期間である。

In [19]:
url1 =  'https://api-tokyochallenge.odpt.org/api/v4/files/DCM-IM/data/02_openData_age_gender_{}.csv?acl:consumerKey={}'
df = pd.read_csv(url1.format('20190422',consumerKey))

HTTPError: HTTP Error 404: The specified blob does not exist.

In [3]:
df_2019.head()

,meshid,lon_center,lat_center,lon_max,lat_max,lon_min,lat_min,pcode,citycode
0,303650061001,136.075317,20.416875,136.075622,20.417083,136.074996,20.416666,13,13421
1,303650061002,136.075942,20.416875,136.076248,20.417083,136.075622,20.416666,13,13421
2,303650061003,136.075317,20.417291,136.075622,20.417499,136.074996,20.417083,13,13421
3,303650061004,136.075942,20.417291,136.076248,20.417499,136.075622,20.417083,13,13421
4,303650061011,136.076568,20.416875,136.076873,20.417083,136.076248,20.416666,13,13421


日吉駅のメッシュを入手しよう。日吉駅の緯度経度は`35.553443,139.64885`。これに近いメッシュを探してみる。

In [18]:
mask = (df_2019['lat_center'] > 35.55) & (df_2019['lat_center'] < 35.56) & (df_2019['lon_center'] > 139.40) & (df_2019['lon_center'] < 139.55)
df_2019[mask]

,meshid,lon_center,lat_center,lon_max,lat_max,lon_min,lat_min,pcode,citycode
305030,533923634242,139.424682,35.556041,139.425003,35.556251,139.424377,35.555831,13,13209
305031,533923634243,139.424057,35.556457,139.424377,35.556667,139.423751,35.556251,13,13209
305032,533923634244,139.424682,35.556457,139.425003,35.556667,139.424377,35.556251,13,13209
305033,533923634341,139.424057,35.556873,139.424377,35.557083,139.423751,35.556667,13,13209
305034,533923634342,139.424682,35.556873,139.425003,35.557083,139.424377,35.556667,13,13209
...,...,...,...,...,...,...,...,...,...
308506,533923781114,139.477188,35.559791,139.477493,35.560001,139.476867,35.559581,13,13209
308507,533923781121,139.477813,35.559375,139.478118,35.559581,139.477493,35.559165,13,13209
308508,533923781122,139.478439,35.559375,139.478744,35.559581,139.478118,35.559165,13,13209
308509,533923781123,139.477813,35.559791,139.478118,35.560001,139.477493,35.559581,13,13209


In [ ]:
summary_url = 'https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/Summary.csv?acl:consumerKey={}'
df_summary = pd.read_csv(summary_url.format(consumerKey))

In [ ]:
df_summary.date = [datetime.strptime(_d, '%Y/%m/%d') for _d in df_summary.date]
plt.figure(figsize=(16,8), dpi=80)
plt.subplot(121)
sns.lineplot(x='date', y='dailyid_count', 
             data=df_summary,
             estimator=None
            )
plt.title('Daily Id count')
plt.subplot(122)
sns.lineplot(x='date', y='log_count', 
             data=df_summary,
             estimator=None
            )
plt.title('Daily log count')
plt.show()

* The range of the data is from 2018/10/01 to 2019/09/30 (365 days).

* There seems to be a sharp drop in data around New Year, the beggining of May, and August. Could this be due to the holidays? 

* What is causing the sharp rise in the amount of users in March and September? 

* From the plot it seems that the data from June to August seems relatively stable. There are no sudden increase or decrease of id counts during this period. Thus we will gather data from `2019/06/01` to `2019/7/7`.

In [4]:
citymaster_url = "https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/prefcode_citycode_master_UTF-8.csv?acl:consumerKey={}"
df_citycode = pd.read_csv(citymaster_url.format(consumerKey))

In [5]:
(df_citycode.set_index(['cityname'])
 .loc[['日吉','三田']]
)

,prefcode,prefname,citycode,cityname,address
0,1,北海道,1101,中央区,北海道札幌市中央区
1,1,北海道,1102,北区,北海道札幌市北区
2,1,北海道,1103,東区,北海道札幌市東区
3,1,北海道,1104,白石区,北海道札幌市白石区
4,1,北海道,1105,豊平区,北海道札幌市豊平区
...,...,...,...,...,...
1892,47,沖縄県,47361,久米島町,沖縄県島尻郡久米島町
1893,47,沖縄県,47362,八重瀬町,沖縄県島尻郡八重瀬町
1894,47,沖縄県,47375,多良間村,沖縄県宮古郡多良間村
1895,47,沖縄県,47381,竹富町,沖縄県八重山郡竹富町


In [ ]:
tokyo_32_dict

In order to further reduce the amount of data we deal with, and also to make the data compatible with wavelet transforms, we limit the number of areas to the 32 districts and cities in Tokyo listed above.

In [ ]:
url = 'https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/PDP_20190721_10.csv?acl:consumerKey='
df_test = pd.read_csv(url + consumerKey)
df_test.head()

We want to count the unique `dailyid` grouped by `citycode` and `gender`. 

In [ ]:
def count_id(df):
    """
    Counts the unique daily id within each city
    """
    return (df.set_index(['citycode'])
            .loc[tokyo_32_dict.values(),['dailyid', 'home_countryname']]
            .reset_index()
            .groupby(['citycode', 'home_countryname'])
            .nunique(['dailyid'])
            .drop(['citycode'], axis=1)
            .reset_index()
           )

In [ ]:
df = (df_test.set_index(['citycode'])
 .loc[tokyo_32_dict.values(), ['dailyid', 'home_countryname']]
 .reset_index()
 .groupby(['citycode', 'home_countryname'])
 .nunique(['dailyid'])
 .drop(['citycode', 'home_countryname'], axis=1)
 .reset_index()
)
df.home_countryname.unique()

In [ ]:
(df.set_index('home_countryname')
 .loc['Australia',:]
)

In [ ]:
df_test.columns

We also want to add the date and time in our dataset in order to plot the time series. However this information is contained when we download the dataset from the database. Ideally, we want to automatically generate the datetime strings for a certain range. 

We want to take strings to set our beginning and set how many days worth or data we want to obtain. We can use `data_range` method in python to obtain a range of dates(we can also set the frequency) and then use the `strftime` function in `datetime` library to convert them into the string we want. 

In [ ]:
def append_timestamp(df, date):
    df['datetime']  = date
    return df

Finally, we can use the pandas `pipe` the create a process chain. This makes the code much more readble. 

In [ ]:
(df_test
 .pipe(count_id_by_city)
 .pipe(append_timestamp, '2019-07-21')
 .head()
)

In [ ]:
def get_data(start: str, days: int):
    """
    start: starting date in 'yyyy-mm-dd' format
    days: Number of days of data we want to obtain 
    """
    url = 'https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/PDP_{}.csv?acl:consumerKey={}'
    consumerKey = 'hnuzMJ1QGE3TcjlYiXohfA6vkGb55OT9xKtTCcd31L4'
    
    periods = days * 24
    df_list = []
    for date in pd.date_range(start=start, periods=periods, freq='H').tolist():
        date_str = datetime.strftime(date, "%Y%m%d_%H")
        try:
            df = (pd.read_csv(url.format(date_str, consumerKey))
                  .pipe(count_id_by_city)
                  .pipe(append_timestamp, date)
                 )
            df_list.append(df)
            
            print('getting data from: {}'.format(date))
            
            time.sleep(1)
        except:
            continue
            
    return pd.concat(df_list)

In [ ]:
df = get_data('2019-06-01', 10)